In [15]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm, AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
penguins = pd.read_csv('/Users/fordfishman/GitHub/envs110/python/data/penguins.csv')
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [3]:
penguins = penguins.loc[(penguins['species'].notnull()) & (penguins['bill_length_mm'].notnull()) & (penguins['sex'].notnull() ),:]

In [14]:
model = ols(formula='bill_length_mm ~ -1 + species*sex + sex + species', data=penguins).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
species,3.0,651492.203043,217164.067681,40501.388850,0.000000e+00
sex,1.0,1135.683888,1135.683888,211.806563,2.422971e-37
species:sex,2.0,24.494427,12.247213,2.284122,1.034865e-01
Residual,327.0,1753.338642,5.361892,NaN,NaN


In [17]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         bill_length_mm   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     305.0
Date:                Tue, 18 Apr 2023   Prob (F-statistic):          9.01e-121
Time:                        11:43:37   Log-Likelihood:                -749.09
No. Observations:                 333   AIC:                             1510.
Df Residuals:                     327   BIC:                             1533.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
species[Adelie]                     37.2575      0.271    137.473      0.000      36.724      37.791
species[Chinstrap]                  46.5735      0.397    117.279      0.000      45.792      47.355
species[Gentoo]                     45.5638      0.304    149.856      0.000      44.966      46.162
sex[T.male]                          3.1329      0.383      8.174      0.000       2.379       3.887
species[T.Chinstrap]:sex[T.male]     1.3877      0.680      2.041      0.042       0.050       2.725
species[T.Gentoo]:sex[T.male]        0.7771      0.572      1.358      0.175      -0.348       1.902
==============================================================================
Omnibus:                       42.630   Durbin-Watson:                   1.908
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.154
Skew:                           0.630   Prob(JB):                     6.57e-23
Kurtosis:                       5.403   Cond. No.                         6.02
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
model.params

species[Adelie]                     37.257534
species[Chinstrap]                  46.573529
species[Gentoo]                     45.563793
sex[T.male]                          3.132877
species[T.Chinstrap]:sex[T.male]     1.387712
species[T.Gentoo]:sex[T.male]        0.777101
dtype: float64

In [10]:
tukey = pairwise_tukeyhsd(endog=penguins['bill_length_mm'], groups=penguins['sex'], alpha=0.05)

print(tukey)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj lower upper  reject
------------------------------------------------
female   male   3.7578   0.0 2.649 4.8666   True
------------------------------------------------


In [16]:
AnovaRM(data=penguins, depvar='bill_length_mm', subject='island', within=['species']).fit()

ValueError: The data set contains more than one observation per subject and cell. Either aggregate the data manually, or pass the `aggregate_func` parameter.